In [8]:
import numpy as np  
import pandas as pd
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import torch, numpy as np
# import seaborn as sns
import sys
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from datasets import Dataset, load_dataset
import pandas as pd 
import math
import numpy as np
import torch, numpy as np
from tqdm.auto import tqdm
from sklearn.cluster import DBSCAN
import shutil
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, adjusted_rand_score
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import os
import json
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score, average_precision_score
import numpy as np

In [3]:
OUTPUT_DIR_GMAP = "/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model/GMAP"
OUTPUT_DIR_MOBILE = "/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model/MOBILE"
OUTPUT_DIR_sample_type = "/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model/Sample_type"
OUTPUT_DIR_age = "/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model/age"

DATA_DIR = "/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/sentences_with_labels"
TSV_FILE_GMAP = f"{DATA_DIR}/gmap_sentences_and_labels.tsv"
TSV_FILE_MOBILE = f"{DATA_DIR}/mobile_sentences_and_labels.tsv"
SEED = 42
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [41]:
def prepare_test_data_only(tsv_file_path, text_column='Sentence', label_column='Sample_type',
                           metadata_columns=['Filename', 'Age', 'Sample_type'],
                           existing_label2id=None):
    """
    Prepare only test data using existing label mappings (for when you have a pre-trained model).

    Args:
        tsv_file_path (str): Path to the test TSV file
        text_column (str): Name of the column containing text data
        label_column (str): Name of the column containing labels
        metadata_columns (list): List of column names to extract as metadata
        existing_label2id (dict): Existing label to ID mapping from training

    Returns:
        tuple: (test_texts, test_labels, metadata_df, unknown_labels, filtered_label2id)
    """
    print(f"Preparing test data from: {tsv_file_path}")

    # Load data
    df = pd.read_csv(tsv_file_path, sep='\t')
    print(f"Loaded {len(df)} test samples")

    # Clean data (remove rows with missing text/label)
    df_clean = df.dropna(subset=[text_column, label_column]).reset_index(drop=True)
    print(f"Test samples after cleaning: {len(df_clean)}")

    unknown_labels = []

    if existing_label2id is not None:
        # Sets of test labels and known labels
        unique_test_labels = set(df_clean[label_column].unique())
        known_labels = set(existing_label2id.keys())

        print("Labels from test data:", unique_test_labels)
        print("Known labels from model:", known_labels)

        # (1) Unknown labels in test (not in known_labels)
        unknown_labels = list(unique_test_labels - known_labels)
        if unknown_labels:
            print(f"\n Warning: Found unknown labels in test data: {unknown_labels}")
            print("These samples will be excluded from evaluation.")
            df_clean = df_clean[df_clean[label_column].isin(known_labels)].reset_index(drop=True)
            print(f"Remaining test samples after filtering unknowns: {len(df_clean)}")

        # (2) Known labels not in test (remove from label2id)
        unused_known_labels = known_labels - unique_test_labels
        if unused_known_labels:
            print(f"\nℹ️ Removing unused labels not in test data: {unused_known_labels}")

        # Filter label2id
        filtered_label2id = {
            label: idx for label, idx in existing_label2id.items()
            if label in unique_test_labels
        }

    else:
        # No existing mapping – build one from test data
        unique_labels = sorted(df_clean[label_column].unique())
        filtered_label2id = {label: i for i, label in enumerate(unique_labels)}

    # Extract texts and label indices
    test_texts = df_clean[text_column].tolist()
    test_labels = [filtered_label2id[label] for label in df_clean[label_column].tolist()]

    # Extract metadata
    available_metadata_cols = [col for col in metadata_columns if col in df_clean.columns]
    metadata_df = df_clean[available_metadata_cols].copy()

    # Print label distribution
    print("\n✅ Test data prepared:")
    print(f"  Samples: {len(test_texts)}")
    print(f"  Label distribution:")
    id2label = {v: k for k, v in filtered_label2id.items()}
    test_label_names = [id2label[label] for label in test_labels]
    print(pd.Series(test_label_names).value_counts())
    
    return test_texts, test_labels, metadata_df, unknown_labels


In [42]:
# Dataset class for handling tokenized data
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [43]:
# def predict_on_test_data(model_path,tokenizer_path, test_tsv_path, existing_label2id, OUTPUT_DIR_PATH,
#                         text_column='Sentence', label_column='Sample_type',
#                         batch_size=16, max_length=512):
#     """
#     Perform predictions on test data using a pre-trained model.
    
#     Args:
#         model_path (str): Path to the saved model
#         test_tsv_path (str): Path to the test TSV file
#         existing_label2id (dict): Label to ID mapping from training
#         text_column (str): Name of text column
#         label_column (str): Name of label column
#         batch_size (int): Batch size for inference
#         max_length (int): Maximum sequence length
    
#     Returns:
#         dict: Dictionary containing predictions, true labels, and metrics
#     """
    
#     # Step 1: Prepare test data
#     print("=== Preparing Test Data ===")
#     test_texts, test_labels, metadata_df, unknown_labels = prepare_test_data_only(
#         test_tsv_path, 
#         text_column=text_column,
#         label_column=label_column,
#         existing_label2id=existing_label2id
#     )
    
#     # Step 2: Load model and tokenizer
#     print("\n=== Loading Model ===")
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     print(f"Using device: {device}")
    
#     tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
#     model = AutoModelForSequenceClassification.from_pretrained(model_path)
#     model.to(device)
#     model.eval()
    
#     # Step 3: Create dataset and dataloader
#     print("\n=== Creating DataLoader ===")
#     test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_length)
#     test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
#     # Step 4: Make predictions
#     print("\n=== Making Predictions ===")
#     predictions = []
#     true_labels = []
#     prediction_probs = []
    
#     with torch.no_grad():
#         for batch in test_dataloader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
            
#             outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#             logits = outputs.logits
            
#             # Get predictions
#             preds = torch.argmax(logits, dim=-1)
#             probs = torch.softmax(logits, dim=-1)
            
#             predictions.extend(preds.cpu().numpy())
#             true_labels.extend(labels.cpu().numpy())
#             prediction_probs.extend(probs.cpu().numpy())
    
#     # Step 5: Convert predictions back to label names
#     id2label = {v: k for k, v in existing_label2id.items()}
#     pred_labels = [id2label[pred] for pred in predictions]
#     true_label_names = [id2label[true] for true in true_labels]
    
#     # Step 6: Calculate metrics
#     print("\n=== Evaluation Results ===")
#     accuracy = accuracy_score(true_labels, predictions)
#     print(f"Accuracy: {accuracy:.4f}")
    
#     print("\nClassification Report:")
#     print(classification_report(true_label_names, pred_labels))
    
#     print("\nConfusion Matrix:")
#     print(confusion_matrix(true_label_names, pred_labels))

#     report_dict = classification_report(true_label_names, pred_labels, output_dict=True)
#     accuracy = accuracy_score(true_label_names, pred_labels)
#     summary_results = [
#     {'metric': 'accuracy', 'class': 'overall', 'value': accuracy},
#     {'metric': 'precision', 'class': 'overall', 'value': report_dict['macro avg']['precision']},
#     {'metric': 'recall', 'class': 'overall', 'value': report_dict['macro avg']['recall']},
#     {'metric': 'f1-score', 'class': 'overall', 'value': report_dict['macro avg']['f1-score']}
# ]

#     # summary_results = []
#     # summary_results.append({
#     #     'metric': 'accuracy',
#     #     'class': 'overall',
#     #     'value': accuracy
#     # })
    
#     for class_name, metrics in report_dict.items():
#         if class_name not in ['accuracy', 'macro avg', 'weighted avg']:
#             summary_results.extend([
#                 {'metric': 'precision', 'class': class_name, 'value': metrics['precision']},
#                 {'metric': 'recall', 'class': class_name, 'value': metrics['recall']},
#                 {'metric': 'f1-score', 'class': class_name, 'value': metrics['f1-score']},
#                 {'metric': 'support', 'class': class_name, 'value': metrics['support']}
#             ])
    
#     summary_df = pd.DataFrame(summary_results)
#     summary_file = os.path.join(OUTPUT_DIR_PATH, f"_evaluation_summary.csv")
#     summary_df.to_csv(summary_file, index=False)
    
    
#     # Step 7: Create results dataframe
#     results_df = metadata_df.copy()
#     results_df['text'] = test_texts
#     results_df['true_label'] = true_label_names
#     results_df['predicted_label'] = pred_labels
#     results_df['prediction_confidence'] = [max(probs) for probs in prediction_probs]
#     results_df['correct'] = results_df['true_label'] == results_df['predicted_label']
    
#     return {
#         'predictions': predictions,
#         'true_labels': true_labels,
#         'pred_labels': pred_labels,
#         'true_label_names': true_label_names,
#         'prediction_probs': prediction_probs,
#         'results_df': results_df,
#         'accuracy': accuracy,
#         'unknown_labels': unknown_labels
#     }

In [44]:
def predict_on_test_data(model_path,tokenizer_path, test_tsv_path, existing_label2id, OUTPUT_DIR_PATH,
                        text_column='Sentence', label_column='Sample_type',
                        batch_size=16, max_length=512):
    """
    Perform predictions on test data using a pre-trained model.
    
    Args:
        model_path (str): Path to the saved model
        test_tsv_path (str): Path to the test TSV file
        existing_label2id (dict): Label to ID mapping from training
        text_column (str): Name of text column
        label_column (str): Name of label column
        batch_size (int): Batch size for inference
        max_length (int): Maximum sequence length
    
    Returns:
        dict: Dictionary containing predictions, true labels, and metrics
    """
    
    # Step 1: Prepare test data
    print("=== Preparing Test Data ===")
    test_texts, test_labels, metadata_df, unknown_labels = prepare_test_data_only(
        test_tsv_path, 
        text_column=text_column,
        label_column=label_column,
        existing_label2id=existing_label2id
    )
    
    # Step 2: Load model and tokenizer
    print("\n=== Loading Model ===")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    model.to(device)
    model.eval()
    
    # Step 3: Create dataset and dataloader
    print("\n=== Creating DataLoader ===")
    test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_length)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Step 4: Make predictions
    print("\n=== Making Predictions ===")
    predictions = []
    true_labels = []
    prediction_probs = []
    
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            
            # Get predictions
            preds = torch.argmax(logits, dim=-1)
            probs = torch.softmax(logits, dim=-1)
            
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            prediction_probs.extend(probs.cpu().numpy())
    
    # Step 5: Convert predictions back to label names
    id2label = {v: k for k, v in existing_label2id.items()}
    pred_labels = [id2label[pred] for pred in predictions]
    true_label_names = [id2label[true] for true in true_labels]
    
    # Step 6: Calculate metrics
    
    print("\n=== Evaluation Results ===")
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\nClassification Report:")
    print(classification_report(true_label_names, pred_labels))
    
    print("\nConfusion Matrix:")
    print(confusion_matrix(true_label_names, pred_labels))

    # Calculate AUC and AUPR
    prediction_probs_array = np.array(prediction_probs)
    
    try:
        num_classes_in_test = len(np.unique(true_labels))
        num_prob_columns = prediction_probs_array.shape[1]
        
        print(f"Debug: Classes in test set: {num_classes_in_test}, Probability columns: {num_prob_columns}")
        
        # For binary classification
        if num_classes_in_test == 2 and num_prob_columns >= 2:
            auc_score = roc_auc_score(true_labels, prediction_probs_array[:, 1])  # Use probability of positive class
            aupr_score = average_precision_score(true_labels, prediction_probs_array[:, 1])
        elif num_classes_in_test > 2:
            # For multi-class - use labels parameter to specify which classes to include
            unique_labels = np.unique(true_labels)
            if len(unique_labels) == num_prob_columns:
                # All classes present in test set
                auc_score = roc_auc_score(true_labels, prediction_probs_array, multi_class='ovr', average='macro')
                aupr_score = average_precision_score(true_labels, prediction_probs_array, average='macro')
            else:
                # Some classes missing from test set - use label binarization
                from sklearn.preprocessing import label_binarize
                from sklearn.metrics import roc_auc_score
                
                # Get all possible labels from the model (0 to num_prob_columns-1)
                all_labels = list(range(num_prob_columns))
                
                # Binarize true labels for all possible classes
                true_labels_binarized = label_binarize(true_labels, classes=all_labels)
                
                # Calculate AUC for each class and average
                auc_scores = []
                for i in range(num_prob_columns):
                    if i in unique_labels:  # Only calculate for classes present in test set
                        class_auc = roc_auc_score(true_labels_binarized[:, i], prediction_probs_array[:, i])
                        auc_scores.append(class_auc)
                
                auc_score = np.mean(auc_scores) if auc_scores else None
                
                # For AUPR, we'll calculate it differently
                aupr_scores = []
                for i in range(num_prob_columns):
                    if i in unique_labels:
                        class_aupr = average_precision_score(true_labels_binarized[:, i], prediction_probs_array[:, i])
                        aupr_scores.append(class_aupr)
                
                aupr_score = np.mean(aupr_scores) if aupr_scores else None
        else:
            print("Warning: Cannot calculate AUC/AUPR - insufficient classes or probability columns")
            auc_score = None
            aupr_score = None
        
        if auc_score is not None:
            print(f"\nAUC (ROC): {auc_score:.4f}")
        if aupr_score is not None:
            print(f"AUPR (Average Precision): {aupr_score:.4f}")
        
    except Exception as e:
        print(f"\nWarning: Could not calculate AUC/AUPR - {e}")
        auc_score = None
        aupr_score = None

    report_dict = classification_report(true_label_names, pred_labels, output_dict=True)
    accuracy = accuracy_score(true_label_names, pred_labels)
    summary_results = [
        {'metric': 'accuracy', 'class': 'overall', 'value': accuracy},
        {'metric': 'precision', 'class': 'overall', 'value': report_dict['macro avg']['precision']},
        {'metric': 'recall', 'class': 'overall', 'value': report_dict['macro avg']['recall']},
        {'metric': 'f1-score', 'class': 'overall', 'value': report_dict['macro avg']['f1-score']}
    ]
    
    # Add AUC and AUPR to summary results if available
    if auc_score is not None:
        summary_results.append({'metric': 'auc', 'class': 'overall', 'value': auc_score})
    if aupr_score is not None:
        summary_results.append({'metric': 'aupr', 'class': 'overall', 'value': aupr_score})

    for class_name, metrics in report_dict.items():
        if class_name not in ['accuracy', 'macro avg', 'weighted avg']:
            summary_results.extend([
                {'metric': 'precision', 'class': class_name, 'value': metrics['precision']},
                {'metric': 'recall', 'class': class_name, 'value': metrics['recall']},
                {'metric': 'f1-score', 'class': class_name, 'value': metrics['f1-score']},
                {'metric': 'support', 'class': class_name, 'value': metrics['support']}
            ])
    
    summary_df = pd.DataFrame(summary_results)
    summary_file = os.path.join(OUTPUT_DIR_PATH, f"_evaluation_summary.csv")
    summary_df.to_csv(summary_file, index=False)
    
    
    # Step 7: Create results dataframe
    results_df = metadata_df.copy()
    results_df['text'] = test_texts
    results_df['true_label'] = true_label_names
    results_df['predicted_label'] = pred_labels
    results_df['prediction_confidence'] = [max(probs) for probs in prediction_probs]
    results_df['correct'] = results_df['true_label'] == results_df['predicted_label']
    
    return {
        'predictions': predictions,
        'true_labels': true_labels,
        'pred_labels': pred_labels,
        'true_label_names': true_label_names,
        'prediction_probs': prediction_probs,
        'results_df': results_df,
        'accuracy': accuracy,
        'unknown_labels': unknown_labels
    }

In [45]:
##### MOBILE ####

In [46]:
model_path = f"{OUTPUT_DIR_sample_type}/model"
tokenizer_path = f"{OUTPUT_DIR_sample_type}/tokenizer"
test_file_path = TSV_FILE_MOBILE

with open(f"{OUTPUT_DIR_sample_type}/label_mappings.json", "r") as f:
    mappings = json.load(f)

label2id = mappings["label2id"]
id2label = mappings["id2label"]
target_label = mappings["target_label"]



# Run predictions
results = predict_on_test_data(
    model_path=model_path,
    tokenizer_path = tokenizer_path,
    test_tsv_path=test_file_path,
    existing_label2id=label2id,
    OUTPUT_DIR_PATH=OUTPUT_DIR_MOBILE,
    text_column='Sentence',
    label_column='Sample_type',
    batch_size=16
)

# Save results
results['results_df'].to_csv(f"{OUTPUT_DIR_MOBILE}/test_predictions.csv", index=False)
print(f"\nResults saved to test_predictions.csv")

# Show some example predictions
print("\n=== Sample Predictions ===")
sample_results = results['results_df'].head(10)
for idx, row in sample_results.iterrows():
    print(f"Text: {row['text'][:100]}...")
    print(f"True: {row['true_label']}, Predicted: {row['predicted_label']}, "
          f"Confidence: {row['prediction_confidence']:.4f}, "
          f"Correct: {row['correct']}")
    print("-" * 50)

=== Preparing Test Data ===
Preparing test data from: /sci/backup/morani/lab/Projects/Aluma/ANLP/Project/sentences_with_labels/mobile_sentences_and_labels.tsv
Loaded 1509 test samples
Test samples after cleaning: 1509
Labels from test data: {'infant_oral', 'vaginal', 'adult_gut', 'skin', 'adult_oral', 'infant_gut'}
Known labels from model: {'adult_gut', 'child_gut', 'vaginal', 'infant_gut'}

These samples will be excluded from evaluation.
Remaining test samples after filtering unknowns: 1178

ℹ️ Removing unused labels not in test data: {'child_gut'}

✅ Test data prepared:
  Samples: 1178
  Label distribution:
infant_gut    454
vaginal       368
adult_gut     356
Name: count, dtype: int64

=== Loading Model ===
Using device: cuda

=== Creating DataLoader ===

=== Making Predictions ===

=== Evaluation Results ===
Accuracy: 0.6995

Classification Report:
              precision    recall  f1-score   support

   adult_gut       0.98      0.15      0.26       356
  infant_gut       0.82   

In [125]:
#### comparison to other models: ####

In [151]:
# def predict_on_test_data_comparisons(model,tokenizer, test_tsv_path, existing_label2id, OUTPUT_DIR_PATH, name,
#                         text_column='Sentence', label_column='Sample_type',
#                         batch_size=16, max_length=512):    
#     # Step 1: Prepare test data
#     print("=== Preparing Test Data ===")
#     test_texts, test_labels, metadata_df, unknown_labels = prepare_test_data_only(
#         test_tsv_path, 
#         text_column=text_column,
#         label_column=label_column,
#         existing_label2id=existing_label2id
#     )
    
#     # Step 2: Load model and tokenizer
#     print("\n=== Loading Model ===")
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     print(f"Using device: {device}")
    
#     model.to(device)
#     model.eval()
    
#     # Step 3: Create dataset and dataloader
#     print("\n=== Creating DataLoader ===")
#     test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_length)
#     test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
#     # Step 4: Make predictions
#     print("\n=== Making Predictions ===")
#     predictions = []
#     true_labels = []
#     prediction_probs = []
    
#     with torch.no_grad():
#         for batch in test_dataloader:
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
            
#             outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#             logits = outputs.logits
            
#             # Get predictions
#             preds = torch.argmax(logits, dim=-1)
#             probs = torch.softmax(logits, dim=-1)
            
#             predictions.extend(preds.cpu().numpy())
#             true_labels.extend(labels.cpu().numpy())
#             prediction_probs.extend(probs.cpu().numpy())
    
#     # Step 5: Convert predictions back to label names
#     id2label = {v: k for k, v in existing_label2id.items()}
#     pred_labels = [id2label[pred] for pred in predictions]
#     true_label_names = [id2label[true] for true in true_labels]
    
#     # Step 6: Calculate metrics
#     print("\n=== Evaluation Results ===")
#     accuracy = accuracy_score(true_labels, predictions)
#     print(f"Accuracy: {accuracy:.4f}")
    
#     print("\nClassification Report:")
#     print(classification_report(true_label_names, pred_labels))
    
#     print("\nConfusion Matrix:")
#     print(confusion_matrix(true_label_names, pred_labels))

#     report_dict = classification_report(true_label_names, pred_labels, output_dict=True)

#     accuracy = accuracy_score(true_label_names, pred_labels)
#     summary_results = [
#     {'metric': 'accuracy', 'class': 'overall', 'value': accuracy},
#     {'metric': 'precision', 'class': 'overall', 'value': report_dict['macro avg']['precision']},
#     {'metric': 'recall', 'class': 'overall', 'value': report_dict['macro avg']['recall']},
#     {'metric': 'f1-score', 'class': 'overall', 'value': report_dict['macro avg']['f1-score']}
# ]
    
#     for class_name, metrics in report_dict.items():
#         if class_name not in ['accuracy', 'macro avg', 'weighted avg']:
#             summary_results.extend([
#                 {'metric': 'precision', 'class': class_name, 'value': metrics['precision']},
#                 {'metric': 'recall', 'class': class_name, 'value': metrics['recall']},
#                 {'metric': 'f1-score', 'class': class_name, 'value': metrics['f1-score']},
#                 {'metric': 'support', 'class': class_name, 'value': metrics['support']}
#             ])
    
#     summary_df = pd.DataFrame(summary_results)
#     summary_file = os.path.join(OUTPUT_DIR_PATH, f"{name}_evaluation_summary.csv")
#     summary_df.to_csv(summary_file, index=False)
    
    
#     # Step 7: Create results dataframe
#     results_df = metadata_df.copy()
#     results_df['text'] = test_texts
#     results_df['true_label'] = true_label_names
#     results_df['predicted_label'] = pred_labels
#     results_df['prediction_confidence'] = [max(probs) for probs in prediction_probs]
#     results_df['correct'] = results_df['true_label'] == results_df['predicted_label']
    
#     return {
#         'predictions': predictions,
#         'true_labels': true_labels,
#         'pred_labels': pred_labels,
#         'true_label_names': true_label_names,
#         'prediction_probs': prediction_probs,
#         'results_df': results_df,
#         'accuracy': accuracy,
#         'unknown_labels': unknown_labels
#     }


In [89]:
def predict_on_test_data_comparisons(model,tokenizer, test_tsv_path, existing_label2id, OUTPUT_DIR_PATH, name,
                        text_column='Sentence', label_column='Sample_type',
                        batch_size=16, max_length=512):    
    # Step 1: Prepare test data
    print("=== Preparing Test Data ===")
    test_texts, test_labels, metadata_df, unknown_labels = prepare_test_data_only(
        test_tsv_path, 
        text_column=text_column,
        label_column=label_column,
        existing_label2id=existing_label2id
    )
    
    # Step 2: Load model and tokenizer
    print("\n=== Loading Model ===")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    model.to(device)
    model.eval()
    
    # Step 3: Create dataset and dataloader
    print("\n=== Creating DataLoader ===")
    test_dataset = TextDataset(test_texts, test_labels, tokenizer, max_length)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Step 4: Make predictions
    print("\n=== Making Predictions ===")
    predictions = []
    true_labels = []
    prediction_probs = []
    
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            
            # Get predictions
            preds = torch.argmax(logits, dim=-1)
            probs = torch.softmax(logits, dim=-1)
            
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
            prediction_probs.extend(probs.cpu().numpy())
    
    # Step 5: Convert predictions back to label names
    id2label = {v: k for k, v in existing_label2id.items()}
    pred_labels = [id2label[pred] for pred in predictions]
    true_label_names = [id2label[true] for true in true_labels]
    
    # Step 6: Calculate metrics
    
    print("\n=== Evaluation Results ===")
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Accuracy: {accuracy:.4f}")
    
    print("\nClassification Report:")
    print(classification_report(true_label_names, pred_labels))
    
    print("\nConfusion Matrix:")
    print(confusion_matrix(true_label_names, pred_labels))

    # Calculate AUC and AUPR
    prediction_probs_array = np.array(prediction_probs)
    
    try:
        num_classes_in_test = len(np.unique(true_labels))
        num_prob_columns = prediction_probs_array.shape[1]
        
        print(f"Debug: Classes in test set: {num_classes_in_test}, Probability columns: {num_prob_columns}")
        
        # For binary classification
        if num_classes_in_test == 2 and num_prob_columns >= 2:
            auc_score = roc_auc_score(true_labels, prediction_probs_array[:, 1])  # Use probability of positive class
            aupr_score = average_precision_score(true_labels, prediction_probs_array[:, 1])
        elif num_classes_in_test > 2:
            # For multi-class - use labels parameter to specify which classes to include
            unique_labels = np.unique(true_labels)
            if len(unique_labels) == num_prob_columns:
                # All classes present in test set
                auc_score = roc_auc_score(true_labels, prediction_probs_array, multi_class='ovr', average='macro')
                aupr_score = average_precision_score(true_labels, prediction_probs_array, average='macro')
            else:
                # Some classes missing from test set - use label binarization
                from sklearn.preprocessing import label_binarize
                from sklearn.metrics import roc_auc_score
                
                # Get all possible labels from the model (0 to num_prob_columns-1)
                all_labels = list(range(num_prob_columns))
                
                # Binarize true labels for all possible classes
                true_labels_binarized = label_binarize(true_labels, classes=all_labels)
                
                # Calculate AUC for each class and average
                auc_scores = []
                for i in range(num_prob_columns):
                    if i in unique_labels:  # Only calculate for classes present in test set
                        class_auc = roc_auc_score(true_labels_binarized[:, i], prediction_probs_array[:, i])
                        auc_scores.append(class_auc)
                
                auc_score = np.mean(auc_scores) if auc_scores else None
                
                # For AUPR, we'll calculate it differently
                aupr_scores = []
                for i in range(num_prob_columns):
                    if i in unique_labels:
                        class_aupr = average_precision_score(true_labels_binarized[:, i], prediction_probs_array[:, i])
                        aupr_scores.append(class_aupr)
                
                aupr_score = np.mean(aupr_scores) if aupr_scores else None
        else:
            print("Warning: Cannot calculate AUC/AUPR - insufficient classes or probability columns")
            auc_score = None
            aupr_score = None
        
        if auc_score is not None:
            print(f"\nAUC (ROC): {auc_score:.4f}")
        if aupr_score is not None:
            print(f"AUPR (Average Precision): {aupr_score:.4f}")
        
    except Exception as e:
        print(f"\nWarning: Could not calculate AUC/AUPR - {e}")
        auc_score = None
        aupr_score = None

    report_dict = classification_report(true_label_names, pred_labels, output_dict=True)
    accuracy = accuracy_score(true_label_names, pred_labels)
    summary_results = [
        {'metric': 'accuracy', 'class': 'overall', 'value': accuracy},
        {'metric': 'precision', 'class': 'overall', 'value': report_dict['macro avg']['precision']},
        {'metric': 'recall', 'class': 'overall', 'value': report_dict['macro avg']['recall']},
        {'metric': 'f1-score', 'class': 'overall', 'value': report_dict['macro avg']['f1-score']}
    ]
    
    # Add AUC and AUPR to summary results if available
    if auc_score is not None:
        summary_results.append({'metric': 'auc', 'class': 'overall', 'value': auc_score})
    if aupr_score is not None:
        summary_results.append({'metric': 'aupr', 'class': 'overall', 'value': aupr_score})

    for class_name, metrics in report_dict.items():
        if class_name not in ['accuracy', 'macro avg', 'weighted avg']:
            summary_results.extend([
                {'metric': 'precision', 'class': class_name, 'value': metrics['precision']},
                {'metric': 'recall', 'class': class_name, 'value': metrics['recall']},
                {'metric': 'f1-score', 'class': class_name, 'value': metrics['f1-score']},
                {'metric': 'support', 'class': class_name, 'value': metrics['support']}
            ])
    summary_df = pd.DataFrame(summary_results)
    summary_file = os.path.join(OUTPUT_DIR_PATH, f"{name}_evaluation_summary.csv")
    summary_df.to_csv(summary_file, index=False)
    
    
    # Step 7: Create results dataframe
    results_df = metadata_df.copy()
    results_df['text'] = test_texts
    results_df['true_label'] = true_label_names
    results_df['predicted_label'] = pred_labels
    results_df['prediction_confidence'] = [max(probs) for probs in prediction_probs]
    results_df['correct'] = results_df['true_label'] == results_df['predicted_label']
    
    return {
        'predictions': predictions,
        'true_labels': true_labels,
        'pred_labels': pred_labels,
        'true_label_names': true_label_names,
        'prediction_probs': prediction_probs,
        'results_df': results_df,
        'accuracy': accuracy,
        'unknown_labels': unknown_labels
    }

In [90]:
import json
import os

def run_model_predictions(MODEL, TOKENIZER, OUTPUT_DIR_PATH, name,
                         test_file_path=TSV_FILE_MOBILE, text_column='Sentence', 
                         label_column='Sample_type', batch_size=16):
    
    # Load label mappings from the output directory
    mappings_path = os.path.join(OUTPUT_DIR_sample_type, "label_mappings.json")
    
    if not os.path.exists(mappings_path):
        raise FileNotFoundError(f"Label mappings not found at {mappings_path}")
    
    with open(mappings_path, "r") as f:
        mappings = json.load(f)
    
    label2id = mappings["label2id"]
    id2label = mappings["id2label"]
    target_label = mappings["target_label"]
    
    if test_file_path is None:
        test_file_path = TSV_FILE_MOBILE
    
    # Run predictions
    results = predict_on_test_data_comparisons(
        model=MODEL,
        tokenizer=TOKENIZER,
        test_tsv_path=TSV_FILE_MOBILE,
        existing_label2id=label2id,
        OUTPUT_DIR_PATH=OUTPUT_DIR_PATH,
        name=name,
        text_column='Sentence',
        label_column='Sample_type',
        batch_size=16
    )
    
    # Save results
    output_file = os.path.join(OUTPUT_DIR_PATH, f"{name}.csv")
    results['results_df'].to_csv(output_file, index=False)
    print(f"\nResults saved to {output_file}")
    
    # Show some example predictions
    print("\n=== Sample Predictions ===")
    sample_results = results['results_df'].head(10)
    for idx, row in sample_results.iterrows():
        print(f"Text: {row['text'][:100]}...")
        print(f"True: {row['true_label']}, Predicted: {row['predicted_label']}, "
              f"Confidence: {row['prediction_confidence']:.4f}, "
              f"Correct: {row['correct']}")
        print("-" * 50)
    
    return results


In [91]:
OUTPUT_DIR_MOBILE_comparisons = "/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model/MOBILE/comparisons_results"

In [132]:
## Results of a regular BERT model - with our tokenizer

In [87]:
OUTPUT_DIR_sample_type_BERT = '/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model/Sample_type/Bert/'
model_path = f"{OUTPUT_DIR_sample_type_BERT}/model"
tokenizer_path = f"{OUTPUT_DIR_sample_type_BERT}/tokenizer"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [88]:
results = run_model_predictions(
    MODEL=model,
    TOKENIZER=tokenizer, 
    OUTPUT_DIR_PATH=OUTPUT_DIR_MOBILE_comparisons,
    name="Bert_our_tokenizer"
)

=== Preparing Test Data ===
Preparing test data from: /sci/backup/morani/lab/Projects/Aluma/ANLP/Project/sentences_with_labels/mobile_sentences_and_labels.tsv
Loaded 1509 test samples
Test samples after cleaning: 1509
Labels from test data: {'infant_oral', 'vaginal', 'adult_gut', 'skin', 'adult_oral', 'infant_gut'}
Known labels from model: {'adult_gut', 'child_gut', 'vaginal', 'infant_gut'}

These samples will be excluded from evaluation.
Remaining test samples after filtering unknowns: 1178

ℹ️ Removing unused labels not in test data: {'child_gut'}

✅ Test data prepared:
  Samples: 1178
  Label distribution:
infant_gut    454
vaginal       368
adult_gut     356
Name: count, dtype: int64

=== Loading Model ===
Using device: cuda

=== Creating DataLoader ===

=== Making Predictions ===
['infant_gut', 'vaginal', 'vaginal', 'infant_gut', 'adult_gut', 'vaginal', 'infant_gut', 'vaginal', 'vaginal', 'infant_gut', 'vaginal', 'infant_gut', 'vaginal', 'infant_gut', 'vaginal', 'vaginal', 'vagina

In [ ]:
## Results of a model with only species level

In [95]:
OUTPUT_DIR_SPECIES_LEVEL = "/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model_species_level/Sample_type"

model_path = f"{OUTPUT_DIR_SPECIES_LEVEL}/model"
tokenizer_path = f"{OUTPUT_DIR_SPECIES_LEVEL}/tokenizer"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [96]:
results = run_model_predictions(
    MODEL=model,
    TOKENIZER=tokenizer, 
    OUTPUT_DIR_PATH=OUTPUT_DIR_MOBILE_comparisons,
    name="species_level_model_our_tokenizer"
)

=== Preparing Test Data ===
Preparing test data from: /sci/backup/morani/lab/Projects/Aluma/ANLP/Project/sentences_with_labels/mobile_sentences_and_labels.tsv
Loaded 1509 test samples
Test samples after cleaning: 1509
Labels from test data: {'infant_oral', 'vaginal', 'adult_gut', 'skin', 'adult_oral', 'infant_gut'}
Known labels from model: {'adult_gut', 'child_gut', 'vaginal', 'infant_gut'}

These samples will be excluded from evaluation.
Remaining test samples after filtering unknowns: 1178

ℹ️ Removing unused labels not in test data: {'child_gut'}

✅ Test data prepared:
  Samples: 1178
  Label distribution:
infant_gut    454
vaginal       368
adult_gut     356
Name: count, dtype: int64

=== Loading Model ===
Using device: cuda

=== Creating DataLoader ===

=== Making Predictions ===

=== Evaluation Results ===
Accuracy: 0.5891

Classification Report:
              precision    recall  f1-score   support

   adult_gut       0.00      0.00      0.00       356
  infant_gut       0.62   

/sci/labs/morani/morani/icore-data/lab/Tools/personal_condas/chen/miniforge3/envs/anlp_project_jupyter_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/sci/labs/morani/morani/icore-data/lab/Tools/personal_condas/chen/miniforge3/envs/anlp_project_jupyter_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/sci/labs/morani/morani/icore-data/lab/Tools/personal_condas/chen/miniforge3/envs/anlp_project_jupyter_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:156

In [166]:
# Biomed Normal tokenizer 

In [93]:
MODEL_NAME_BIOMED = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext"
OUTPUT_DIR_sample_type_BIOMED = "/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model/Sample_type/Biomed"

model_path = f"{OUTPUT_DIR_sample_type_BIOMED}/model"
tokenizer_path = f"{OUTPUT_DIR_sample_type_BIOMED}/tokenizer"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [94]:
results = run_model_predictions(
    MODEL=model,
    TOKENIZER=tokenizer, 
    OUTPUT_DIR_PATH=OUTPUT_DIR_MOBILE_comparisons,
    name="Biomed_model_regular_tokenizer"
)

=== Preparing Test Data ===
Preparing test data from: /sci/backup/morani/lab/Projects/Aluma/ANLP/Project/sentences_with_labels/mobile_sentences_and_labels.tsv
Loaded 1509 test samples
Test samples after cleaning: 1509
Labels from test data: {'infant_oral', 'vaginal', 'adult_gut', 'skin', 'adult_oral', 'infant_gut'}
Known labels from model: {'adult_gut', 'child_gut', 'vaginal', 'infant_gut'}

These samples will be excluded from evaluation.
Remaining test samples after filtering unknowns: 1178

ℹ️ Removing unused labels not in test data: {'child_gut'}

✅ Test data prepared:
  Samples: 1178
  Label distribution:
infant_gut    454
vaginal       368
adult_gut     356
Name: count, dtype: int64

=== Loading Model ===
Using device: cuda

=== Creating DataLoader ===

=== Making Predictions ===

=== Evaluation Results ===
Accuracy: 0.7029

Classification Report:
              precision    recall  f1-score   support

   adult_gut       0.98      0.17      0.29       356
  infant_gut       0.77   

In [140]:
## results ##

In [97]:
import pandas as pd
import os
import glob
from pathlib import Path

def create_model_comparison_table(results_directory, output_file=None):
    
    pattern = os.path.join(results_directory, "*_evaluation_summary.csv")
    csv_files = glob.glob(pattern)
    
    if not csv_files:
        print(f"No evaluation summary files found in {results_directory}")
        return None
    
    print(f"Found {len(csv_files)} evaluation files:")
    for file in csv_files:
        print(f"  - {os.path.basename(file)}")
    
    comparison_data = {}
    
    for csv_file in csv_files:
        model_name = os.path.basename(csv_file).replace('_evaluation_summary.csv', '')

        try:
            df = pd.read_csv(csv_file)
            model_metrics = {}

            for _, row in df.iterrows():
                metric_name = row['metric']
                class_name = row['class']
                value = row['value']

                if class_name == 'overall':
                    column_name = metric_name
                else:
                    column_name = f"{class_name}_{metric_name}"
                
                model_metrics[column_name] = value
            
            comparison_data[model_name] = model_metrics
            
        except Exception as e:
            print(f"Error reading {csv_file}: {e}")
            continue
            
    comparison_df = pd.DataFrame.from_dict(comparison_data, orient='index')

    overall_cols = [col for col in comparison_df.columns if not '_' in col or col.startswith(('macro', 'weighted'))]
    class_cols = [col for col in comparison_df.columns if col not in overall_cols]

    overall_cols.sort()
    class_cols.sort()

    ordered_cols = overall_cols + class_cols
    comparison_df = comparison_df[ordered_cols]

    comparison_df = comparison_df.round(4)
    
    print(f"\nComparison table created with shape: {comparison_df.shape}")
    print(f"Models: {list(comparison_df.index)}")
    print(f"Metrics: {len(comparison_df.columns)}")

    if output_file:
        comparison_df.to_csv(output_file)
        print(f"Comparison table saved to: {output_file}")
    
    return comparison_df

In [98]:
results_dir = "/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model/MOBILE/comparisons_results/"
    
    # Create full comparison table
full_comparison = create_model_comparison_table(
    results_directory=results_dir,
    output_file="/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model/MOBILE/comparisons_results/full_model_comparison.csv"
)

Found 4 evaluation files:
  - Bert_our_tokenizer_evaluation_summary.csv
  - Biomed_model_regular_tokenizer_evaluation_summary.csv
  - our_model_evaluation_summary.csv
  - species_level_model_our_tokenizer_evaluation_summary.csv

Comparison table created with shape: (4, 18)
Models: ['Bert_our_tokenizer', 'Biomed_model_regular_tokenizer', 'our_model', 'species_level_model_our_tokenizer']
Metrics: 18
Comparison table saved to: /sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model/MOBILE/comparisons_results/full_model_comparison.csv


In [99]:
results_table = pd.read_csv("/sci/backup/morani/lab/Projects/Aluma/ANLP/Project/supervised_model/MOBILE/comparisons_results/full_model_comparison.csv")

In [100]:
results_table

,Unnamed: 0,accuracy,auc,aupr,f1-score,precision,recall,adult_gut_f1-score,adult_gut_precision,adult_gut_recall,adult_gut_support,infant_gut_f1-score,infant_gut_precision,infant_gut_recall,infant_gut_support,vaginal_f1-score,vaginal_precision,vaginal_recall,vaginal_support
0,Bert_our_tokenizer,0.6825,0.8436,0.7463,0.5991,0.7644,0.6541,0.2663,0.9649,0.1545,356.0,0.8095,0.6928,0.9736,454.0,0.7215,0.6356,0.8342,368.0
1,Biomed_model_regular_tokenizer,0.7029,0.9005,0.8259,0.6213,0.7849,0.6763,0.2878,0.9836,0.1685,356.0,0.8577,0.7692,0.9692,454.0,0.7185,0.6018,0.8913,368.0
2,our_model,0.6995,0.9000,0.8312,0.6133,0.7884,0.6725,0.2585,0.9815,0.1489,356.0,0.8889,0.8209,0.9692,454.0,0.6925,0.5629,0.8995,368.0
3,species_level_model_our_tokenizer,0.5891,0.8772,0.8049,0.4603,0.3907,0.5602,0.0000,0.0000,0.0000,356.0,0.7261,0.6186,0.8789,454.0,0.6548,0.5535,0.8016,368.0


In [ ]:
### GMAP ###

In [ ]:
model_path = f"{OUTPUT_DIR_age}/model"
tokenizer_path = f"{OUTPUT_DIR_age}/tokenizer"
test_file_path = TSV_FILE_GMAP

with open(f"{OUTPUT_DIR_age}/label_mappings.json", "r") as f:
    mappings = json.load(f)

label2id = mappings["label2id"]
id2label = mappings["id2label"]
target_label = mappings["target_label"]


# Run predictions
results = predict_on_test_data(
    model_path=model_path,
    tokenizer_path = tokenizer_path,
    test_tsv_path=test_file_path,
    existing_label2id=label2id,
    text_column='Sentence',
    label_column='Age',
    batch_size=16
)

# Save results
results['results_df'].to_csv(f"{OUTPUT_DIR_GMAP}/test_predictions.csv", index=False)

print(f"\nResults saved to test_predictions.csv")

# Show some example predictions
print("\n=== Sample Predictions ===")
sample_results = results['results_df'].head(10)
for idx, row in sample_results.iterrows():
    print(f"Text: {row['text'][:100]}...")
    print(f"True: {row['true_label']}, Predicted: {row['predicted_label']}, "
          f"Confidence: {row['prediction_confidence']:.4f}, "
          f"Correct: {row['correct']}")
    print("-" * 50)